In [6]:
from sagas.nlu.ruleset_procs import cached_chunks
sents='I want to play music'
lang='en'
chunks = cached_chunks(sents, lang, 'corenlp')
chunks

.. request is {'lang': 'en', 'sents': 'I want to play music', 'engine': 'corenlp', 'pipelines': ['predicts']}


{'doc': <sagas.nlu.uni_jsonifier.JsonifySentImpl at 0x121be2470>,
 'predicts': [],
 'verb_domains': [{'index': '2',
   'text': 'want',
   'lemma': 'want',
   'upos': 'VERB',
   'xpos': 'VBP',
   'feats': 'Mood=Ind|Tense=Pres|VerbForm=Fin',
   'governor': 0,
   'dependency_relation': 'root',
   'xcomp': [{'index': '4',
     'text': 'play',
     'lemma': 'play',
     'upos': 'VERB',
     'xpos': 'VB',
     'feats': 'VerbForm=Inf',
     'governor': 2,
     'dependency_relation': 'xcomp',
     'mark': [{'index': '3',
       'text': 'to',
       'lemma': 'to',
       'upos': 'PART',
       'xpos': 'TO',
       'feats': '_',
       'governor': 4,
       'dependency_relation': 'mark'}],
     'obj': [{'index': '5',
       'text': 'music',
       'lemma': 'music',
       'upos': 'NOUN',
       'xpos': 'NN',
       'feats': 'Number=Sing',
       'governor': 4,
       'dependency_relation': 'obj'}]}],
   'nsubj': [{'index': '1',
     'text': 'I',
     'lemma': 'I',
     'upos': 'PRON',
     'xpos

In [10]:
from sagas.nlu.ruleset_procs import create_host, list_words
import sagas.tracker_fn as tc
import sagas.nlu.ruleset_fixtures as rf
from pprint import pprint

v_domains=chunks['verb_domains']

intents=[]
host = create_host()
verbose=False

for d in v_domains:
    tokens=list_words(d, lang, with_chains=True)
    if verbose:
        pprint(tokens)

    # specs evaluate
    # tc.emp('cyan', f"✁ specs evaluate. {'-' * 25}")
    r3={}
    for token in tokens:
        r3 = host.assert_fact('chains', token)
        pprint(r3) # the last result is all state
    [r3.pop(key) for key in ['$s', 'id', 'sid']]
    # tc.emp('red', f"specs state - {r3}")

    # sents evaluate
    # tc.emp('cyan', f"✁ sents evaluate. {'-' * 25}")
    sents_data={**d, **r3}
    tc.emp('cyan', f"  keys: {', '.join(sents_data.keys())}")
    result=host.assert_fact('sents', sents_data)
    # tc.emp('red', f"sents state - {result}")
    if 'intents' in result:
        intents.extend(result['intents'])
        
intents        

word-> Fact: _ want verb
catalog-> Fact: _ is willing
{'$s': 1, 'id': 'sid-0', 'sid': '0', 'spec_verb': ['willing']}
word-> Fact: _/xcomp play verb
catalog-> Fact: _/xcomp is perform
{'$s': 1,
 'id': 'sid-0',
 'sid': '0',
 'spec_verb': ['willing'],
 'spec_xcomp': ['perform']}
word-> Fact: _/xcomp/obj music noun
catalog-> Fact: _/xcomp/obj as sound
{'$s': 1,
 'id': 'sid-0',
 'sid': '0',
 'spec_verb': ['willing'],
 'spec_xcomp': ['perform'],
 'spec_xcomp_obj': ['sound']}
  keys: index, text, lemma, upos, xpos, feats, governor, dependency_relation, xcomp, nsubj, punct, spec_verb, spec_xcomp, spec_xcomp_obj
catalog-> Fact: nsubj is prop
sents-> Fact: want want


[{'intent': 'perform_media', 'object_type': 'sound'}]

In [13]:
from sagas.nlu.ruleset_actions import ruleset_actions
def get_schedules(sents, lang, intents):
    action_binds = ruleset_actions.get_intents()    
    schedules = []
    for intent_item in intents:
        intent = intent_item['intent']
        acts = [ac['action'] for ac in action_binds if ac['intent'] == intent]

        # tc.emp('green', f"action for intent {intent}: {acts}")
        if len(acts) > 0:
            schedules.append({'intent': intent, 'action': acts,
                              'sents': sents, 'lang': lang,
                              'object_type': intent_item['object_type'],
                              'parameters': {},
                              })
    return schedules

schedules=get_schedules(sents, lang, intents)
schedules

[{'intent': 'perform_media',
  'action': ['action_perform_media'],
  'sents': 'I want to play music',
  'lang': 'en',
  'object_type': 'sound',
  'parameters': {}}]

In [17]:
import requests
from sagas.conf.conf import cf

def comm(schedules):
    for ac in schedules:
        values={"object_type": ac["object_type"],
                "sents":ac["sents"],
                'parameters': ac['parameters']}
        values_str=json.dumps(values, ensure_ascii=False)
        # text = f'/{ac["intent"]}{{"object_type": "{ac["object_type"]}", "sents":"{ac["sents"]}"}}'
        text = f'/{ac["intent"]}{values_str}'
        data = {'mod': 'genesis', 'lang': ac['lang'], "sents": text}
        response = requests.post(f'{cf.ensure("agents_servant")}/message/my', json=data)
        print('status code:', response.status_code)
        pprint(response.json())

comm(schedules)

status code: 200
[{'custom': {'media_list': ['first song', 'second song'],
             'media_type': 'sound',
             'result': 'success',
             'sents': 'I want to play music'},
  'recipient_id': 'my'}]


In [18]:
schedules= [{'intent': 'perform_media',
             'object_type': 'testing',
             'action': ['action_perform_sound'],
             'sents': 'I want to play music.',
             'parameters': {
                'media_list': ['first song', 'second song'],
                'media_type': 'sound'},
             'lang': 'en'}]
comm(schedules)

status code: 200
[{'custom': {'media_list': ['first song', 'second song'],
             'media_type': 'testing',
             'result': 'success',
             'sents': 'I want to play music.'},
  'recipient_id': 'my'}]


In [22]:
# fallback：has-intent>confidence
# ruleset-intents和fallback-intents都在execute阶段获取
from sagas.nlu.rasa_procs import invoke_nlu

endpoint = cf.ensure('nlu_multilang_servant')
print('.. with endpoing', endpoint)
result = invoke_nlu(endpoint, lang, "current", sents)
if result != None and len(result)>0:
    print(json.dumps(result, indent=4, ensure_ascii=False))

    intent = result["intent"]
    print('%s -> %f' % (intent['name'], intent['confidence']))
    entities = result['entities']
    print('entities ->', [ent['entity'] for ent in entities])

.. with endpoing http://localhost:18099
{
    "intent": {
        "name": "inform",
        "confidence": 0.28809934854507446
    },
    "entities": [],
    "intent_ranking": [
        {
            "name": "inform",
            "confidence": 0.28809934854507446
        },
        {
            "name": "chitchat",
            "confidence": 0.13150081038475037
        },
        {
            "name": "deny",
            "confidence": 0.13036714494228363
        },
        {
            "name": "stop",
            "confidence": 0.10581902414560318
        },
        {
            "name": "affirm",
            "confidence": 0.10453446209430695
        },
        {
            "name": "request_restaurant",
            "confidence": 0.058347396552562714
        },
        {
            "name": "bot_challenge",
            "confidence": 0.05678541213274002
        },
        {
            "name": "name",
            "confidence": 0.043068546801805496
        },
        {
            "name": 